# Molecular Descriptors

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Stef0916/chemoinformatics-bioinformatics/blob/main/cheminformatics-workflow/notebooks/5-Molecular-descriptors.ipynb)

Molecular descriptors are numerical values or vectors that represent certain structural features or properties of molecules. RDKit provides a wide range of descriptors, including:

- Topological descriptors
- Geometrical descriptors
- Electronic descriptors
- Physicochemical descriptors

## Content

1. [Import Libraries](#1)
2. [Example from PubChem Dataset](#2)
    - 2.1 [Load Data](#3)
    - 2.2 [Visualization](#4)
3. [Descriptors](#5)
    - 3.1 [MW, LogP, RB Descriptors](#6)
        - 3.1.1 [Individual Descriptor Calculations](#7)
        - 3.1.2 [Define Function](#8)
    - 3.2 [Aromatic Rings, Valence Electrons and BalabanJ descriptors](#9)
    - 3.3 [General RDKit Molecular Descriptors](#10)
4. [Save Dataset](#11)



## 1. Import Libraries<a name = 1></a>

In [1]:
!pip install rdkit

In [2]:
!pip install mols2grid #to visualize in a grid

In [3]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage
from rdkit.Chem import Draw
from rdkit.Chem import rdDepictor
IPythonConsole.ipython_useSVG = True
rdDepictor.SetPreferCoordGen(True)
from rdkit.Chem import Descriptors, PandasTools, AllChem, rdMolDescriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.Descriptors import MolWt, NumRotatableBonds
from rdkit.Chem.Crippen import MolLogP
import mols2grid

import rdkit
rdkit.__version__

'2023.09.1'

## 2. Example from PubChem DataSet<a name = 2></a>

### 2.1 Load Data<a name = 3></a>

In [27]:
data = PandasTools.LoadSDF('AID_1259247_sanitized.sdf', molColName='Molecule')

In [28]:
data.head()

,PUBCHEM_SMILES,PUBCHEM_ACTIVITY_OUTCOME,Antagonist Activity,Viability Activity,Antagonist Efficacy (%),ID,Molecule
0,CN1C2=NC=NC3=C2C(=CN3[C@H]4[C@@H]([C@@H]([C@H]...,Active,active antagonist,inactive,-37.5702,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b840>
1,C[C@]12CC[C@H]3C(=CCC4=C3C=CC(=C4)O)[C@@H]1CCC2=O,Active,active antagonist,inactive,-37.9252,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b760>
2,C1=NC2=C(N=C(N=C2N1[C@H]3[C@H]([C@@H]([C@H](O3...,Active,active antagonist,inactive,-35.847,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b7d0>
3,CCOC1=CC2=C(C3=C(C=C(C=C3)N)N=C2C=C1)N.CC(C(=O...,Active,active antagonist,inactive,-35.5194,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537ca0>
4,CCCCOC(=O)C1=CC=CC=C1C(=O)OCC2=CC=CC=C2,Active,active antagonist,inactive,-39.4007,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537d80>


In [29]:
data['Antagonist Efficacy (%)'].dtype

dtype('O')

In [30]:
data['Antagonist Efficacy (%)'] = data['Antagonist Efficacy (%)'].astype('float64')

In [31]:
data['Antagonist Efficacy (%)'].dtype

dtype('float64')

### 2.2 Visualization<a name = 4></a>

In [32]:
mols2grid.display(data, mol_col = 'Molecule', subset=['Viability Activity', 'Antagonist Efficacy (%)'],
                  transform={"Antagonist Efficacy (%)": lambda x: f"{x:.2f}"})

MolGridWidget()

## 3. Descriptors<a name = 5></a>

### 3.1. MW, LogP and RB descriptors<a name = 6></a>

#### 3.1.1 Individual Descriptor Calculations<a name = 7></a>

In [33]:
mol_weight = [MolWt(mol) for mol in data['Molecule']]
mol_weight = [round(num, 2) for num in mol_weight]
mol_weight[:5]

[320.31, 268.36, 303.68, 253.31, 312.36]

Let's add the MW in our DataFrame

In [34]:
data['Molecular Weight'] = mol_weight

In [35]:
data.head()

,PUBCHEM_SMILES,PUBCHEM_ACTIVITY_OUTCOME,Antagonist Activity,Viability Activity,Antagonist Efficacy (%),ID,Molecule,Molecular Weight
0,CN1C2=NC=NC3=C2C(=CN3[C@H]4[C@@H]([C@@H]([C@H]...,Active,active antagonist,inactive,-37.5702,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b840>,320.31
1,C[C@]12CC[C@H]3C(=CCC4=C3C=CC(=C4)O)[C@@H]1CCC2=O,Active,active antagonist,inactive,-37.9252,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b760>,268.36
2,C1=NC2=C(N=C(N=C2N1[C@H]3[C@H]([C@@H]([C@H](O3...,Active,active antagonist,inactive,-35.8470,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b7d0>,303.68
3,CCOC1=CC2=C(C3=C(C=C(C=C3)N)N=C2C=C1)N.CC(C(=O...,Active,active antagonist,inactive,-35.5194,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537ca0>,253.31
4,CCCCOC(=O)C1=CC=CC=C1C(=O)OCC2=CC=CC=C2,Active,active antagonist,inactive,-39.4007,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537d80>,312.36


In [36]:
logp = [MolLogP(mol) for mol in data['Molecule']]
logp = [round(num, 2) for num in logp]
logp[:5]

[-1.89, 3.9, -0.35, 2.95, 4.0]

In [37]:
data['LogP'] = logp

In [38]:
data.head()

,PUBCHEM_SMILES,PUBCHEM_ACTIVITY_OUTCOME,Antagonist Activity,Viability Activity,Antagonist Efficacy (%),ID,Molecule,Molecular Weight,LogP
0,CN1C2=NC=NC3=C2C(=CN3[C@H]4[C@@H]([C@@H]([C@H]...,Active,active antagonist,inactive,-37.5702,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b840>,320.31,-1.89
1,C[C@]12CC[C@H]3C(=CCC4=C3C=CC(=C4)O)[C@@H]1CCC2=O,Active,active antagonist,inactive,-37.9252,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b760>,268.36,3.90
2,C1=NC2=C(N=C(N=C2N1[C@H]3[C@H]([C@@H]([C@H](O3...,Active,active antagonist,inactive,-35.8470,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b7d0>,303.68,-0.35
3,CCOC1=CC2=C(C3=C(C=C(C=C3)N)N=C2C=C1)N.CC(C(=O...,Active,active antagonist,inactive,-35.5194,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537ca0>,253.31,2.95
4,CCCCOC(=O)C1=CC=CC=C1C(=O)OCC2=CC=CC=C2,Active,active antagonist,inactive,-39.4007,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537d80>,312.36,4.00


In [39]:
num_rot_bonds = [NumRotatableBonds(mol) for mol in data['Molecule']]
num_rot_bonds = [round(num, 2) for num in num_rot_bonds]
num_rot_bonds[:5]

[2, 0, 2, 2, 7]

In [40]:
data['Number Rotatable Bonds'] = num_rot_bonds

In [41]:
data.head()

,PUBCHEM_SMILES,PUBCHEM_ACTIVITY_OUTCOME,Antagonist Activity,Viability Activity,Antagonist Efficacy (%),ID,Molecule,Molecular Weight,LogP,Number Rotatable Bonds
0,CN1C2=NC=NC3=C2C(=CN3[C@H]4[C@@H]([C@@H]([C@H]...,Active,active antagonist,inactive,-37.5702,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b840>,320.31,-1.89,2
1,C[C@]12CC[C@H]3C(=CCC4=C3C=CC(=C4)O)[C@@H]1CCC2=O,Active,active antagonist,inactive,-37.9252,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b760>,268.36,3.90,0
2,C1=NC2=C(N=C(N=C2N1[C@H]3[C@H]([C@@H]([C@H](O3...,Active,active antagonist,inactive,-35.8470,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b7d0>,303.68,-0.35,2
3,CCOC1=CC2=C(C3=C(C=C(C=C3)N)N=C2C=C1)N.CC(C(=O...,Active,active antagonist,inactive,-35.5194,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537ca0>,253.31,2.95,2
4,CCCCOC(=O)C1=CC=CC=C1C(=O)OCC2=CC=CC=C2,Active,active antagonist,inactive,-39.4007,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537d80>,312.36,4.00,7


In [42]:
data.shape

(5134, 10)

#### 3.1.2 Define a function<a name = 8></a>

In [43]:
def calculate_MW_LogP_RB(molecules):
    """
    Calculates molecular weight, LogP, and number of rotatable bonds for a list of RDKit molecule objects.

    Parameters:
    - molecules (iterable): An iterable of RDKit Mol objects.

    Returns:
    - pd.DataFrame: A DataFrame with columns 'Molecular Weight', 'LogP', and 'Rotatable Bonds'.
    """
    mol_weight = []
    logp = []
    num_rot_bonds = []

    # Calculate descriptors for each molecule
    for mol in molecules:
        wt = MolWt(mol)  # Get the molecular weight
        mol_weight.append(round(wt, 2))  # Round the molecular weight to 2 decimal places

        lp = MolLogP(mol)  # Get the LogP value
        logp.append(round(lp, 2))  # Round the LogP to 2 decimal places

        nb = NumRotatableBonds(mol)  # Get the number of rotatable bonds
        num_rot_bonds.append(nb)  # No need to round an integer

    # Create a DataFrame with the calculated descriptors
    descriptors_df = pd.DataFrame({
        'Molecular Weight': mol_weight,
        'LogP': logp,
        'Rotatable Bonds': num_rot_bonds
    })

    return descriptors_df

In [44]:
descriptors_df = calculate_MW_LogP_RB(data['Molecule'])

In [45]:
descriptors_df

,Molecular Weight,LogP,Rotatable Bonds
0,320.31,-1.89,2
1,268.36,3.90,0
2,303.68,-0.35,2
3,253.31,2.95,2
4,312.36,4.00,7
...,...,...,...
5129,110.97,1.98,1
5130,208.24,3.49,6
5131,88.17,1.51,0
5132,209.25,1.51,6


Now, we can add this descriptors to our grid visualization

In [46]:
# From https://github.com/PatWalters/practical_cheminformatics_tutorials/tree/main/fundamentals
two_decimals = lambda x: f"{x:.2f}"
mols2grid.display(data,mol_col="Molecule",subset=["img","Molecular Weight","LogP","Number Rotatable Bonds"],transform={"Molecular Weight" : two_decimals, "LogP": two_decimals})

MolGridWidget()

### 3.2 Aromatic Rings, Valence Electrons and BalabanJ descriptors<a name = 9></a>

In [47]:
# Calculating descriptors bundle

NumAromaticRings = []
NumValenceElectrons = []
BalabanJ = []

for mol in data['Molecule']:
    NumAromaticRings.append(rdMolDescriptors.CalcNumAromaticRings(mol))
    NumValenceElectrons.append(Descriptors.NumValenceElectrons(mol))
    BalabanJ.append(GraphDescriptors.BalabanJ(mol))

BalabanJ = [round(num, 2) for num in BalabanJ]

descr_df = pd.DataFrame({
    'Aromatic Rings': NumAromaticRings,
    'Valence Electrons': NumValenceElectrons,
    'BalabanJ': BalabanJ
})

In [48]:
descr_df

,Aromatic Rings,Valence Electrons,BalabanJ
0,2,122,1.89
1,1,104,1.77
2,2,108,2.07
3,3,96,2.33
4,2,120,2.03
...,...,...,...
5129,0,30,2.62
5130,0,80,3.87
5131,0,30,2.08
5132,1,82,2.31


In [49]:
data = pd.concat([data, descr_df], axis=1)
data.head()

,PUBCHEM_SMILES,PUBCHEM_ACTIVITY_OUTCOME,Antagonist Activity,Viability Activity,Antagonist Efficacy (%),ID,Molecule,Molecular Weight,LogP,Number Rotatable Bonds,Aromatic Rings,Valence Electrons,BalabanJ
0,CN1C2=NC=NC3=C2C(=CN3[C@H]4[C@@H]([C@@H]([C@H]...,Active,active antagonist,inactive,-37.5702,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b840>,320.31,-1.89,2,2,122,1.89
1,C[C@]12CC[C@H]3C(=CCC4=C3C=CC(=C4)O)[C@@H]1CCC2=O,Active,active antagonist,inactive,-37.9252,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b760>,268.36,3.90,0,1,104,1.77
2,C1=NC2=C(N=C(N=C2N1[C@H]3[C@H]([C@@H]([C@H](O3...,Active,active antagonist,inactive,-35.8470,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b7d0>,303.68,-0.35,2,2,108,2.07
3,CCOC1=CC2=C(C3=C(C=C(C=C3)N)N=C2C=C1)N.CC(C(=O...,Active,active antagonist,inactive,-35.5194,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537ca0>,253.31,2.95,2,3,96,2.33
4,CCCCOC(=O)C1=CC=CC=C1C(=O)OCC2=CC=CC=C2,Active,active antagonist,inactive,-39.4007,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537d80>,312.36,4.00,7,2,120,2.03


### 3.3 General Molecular Descriptors<a name = 10></a>

The Descriptors module has a list of the available descriptors. The list is made of 2-tuples (name, function):

In [50]:
# From https://greglandrum.github.io/rdkit-blog/posts/2022-12-23-descriptor-tutorial.html
print(len(Descriptors._descList))
print(Descriptors._descList[:10])

211
[('MaxAbsEStateIndex', <function MaxAbsEStateIndex at 0x7e9719eb1ea0>), ('MaxEStateIndex', <function MaxEStateIndex at 0x7e9719eb1d80>), ('MinAbsEStateIndex', <function MinAbsEStateIndex at 0x7e9719eb1f30>), ('MinEStateIndex', <function MinEStateIndex at 0x7e9719eb1e10>), ('qed', <function qed at 0x7e9717310d30>), ('SPS', <function SPS at 0x7e9717311120>), ('MolWt', <function <lambda> at 0x7e9717311750>), ('HeavyAtomMolWt', <function HeavyAtomMolWt at 0x7e97173117e0>), ('ExactMolWt', <function <lambda> at 0x7e9717311870>), ('NumValenceElectrons', <function NumValenceElectrons at 0x7e9717311900>)]


In [51]:
# From https://github.com/gashawmg/molecular-descriptors
def RDkit_descriptors(data):
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors =[]
    for mol in data:
        # # add hydrogens to molecules
        # mol=Chem.AddHs(mol) #Because there are 3D descriptors like conformes that needs H atoms
        # Calculate all 211 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names

In [52]:
Mol_descriptors,desc_names = RDkit_descriptors(data['Molecule'])

In [53]:
df_with_208_descriptors = pd.DataFrame(Mol_descriptors,columns=desc_names)
df_with_208_descriptors

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,SPS
0,10.234414,10.234414,0.296568,-1.203091,0.503084,30.086957,320.309,304.181,320.123303,122,...,0,0,0,0,0,0,0,0,0,30.086957
1,12.237860,12.237860,0.083404,-0.083404,0.774541,38.250000,268.356,248.196,268.146330,104,...,0,0,0,0,0,0,0,0,0,38.250000
2,14.068550,14.068550,0.061492,-1.736560,0.648650,30.200000,303.681,292.593,303.053445,108,...,0,0,0,0,0,0,0,0,0,30.200000
3,6.233382,6.233382,0.627931,0.627931,0.543636,11.000000,253.305,238.185,253.121512,96,...,0,0,0,0,0,0,0,0,0,11.000000
4,12.243244,12.243244,0.165565,-0.530653,0.572576,10.130435,312.365,292.205,312.136159,120,...,0,0,0,0,0,0,0,1,0,10.130435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5129,5.130401,5.130401,0.503472,0.503472,0.453516,10.000000,110.971,106.939,109.969005,30,...,0,0,0,0,0,0,0,0,0,10.000000
5130,5.492315,5.492315,0.142233,-1.179012,0.625185,12.461538,208.238,187.070,208.122831,80,...,0,0,0,0,0,0,0,0,0,12.461538
5131,2.074074,2.074074,1.416667,1.416667,0.432299,24.000000,88.175,80.111,88.034671,30,...,0,0,0,0,0,0,0,0,0,24.000000
5132,10.053967,10.053967,0.515044,-0.515044,0.569336,12.133333,209.249,194.129,209.116427,82,...,0,0,0,0,0,0,0,0,0,12.133333


## 4. Save Dataset<a name = 11></a>

In [54]:
data_descr = pd.concat([data, df_with_208_descriptors], axis=1)

In [56]:
data_descr.head()

,PUBCHEM_SMILES,PUBCHEM_ACTIVITY_OUTCOME,Antagonist Activity,Viability Activity,Antagonist Efficacy (%),ID,Molecule,Molecular Weight,LogP,Number Rotatable Bonds,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,SPS
0,CN1C2=NC=NC3=C2C(=CN3[C@H]4[C@@H]([C@@H]([C@H]...,Active,active antagonist,inactive,-37.5702,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b840>,320.31,-1.89,2,...,0,0,0,0,0,0,0,0,0,30.086957
1,C[C@]12CC[C@H]3C(=CCC4=C3C=CC(=C4)O)[C@@H]1CCC2=O,Active,active antagonist,inactive,-37.9252,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b760>,268.36,3.90,0,...,0,0,0,0,0,0,0,0,0,38.250000
2,C1=NC2=C(N=C(N=C2N1[C@H]3[C@H]([C@@H]([C@H](O3...,Active,active antagonist,inactive,-35.8470,,<rdkit.Chem.rdchem.Mol object at 0x7e9714f2b7d0>,303.68,-0.35,2,...,0,0,0,0,0,0,0,0,0,30.200000
3,CCOC1=CC2=C(C3=C(C=C(C=C3)N)N=C2C=C1)N.CC(C(=O...,Active,active antagonist,inactive,-35.5194,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537ca0>,253.31,2.95,2,...,0,0,0,0,0,0,0,0,0,11.000000
4,CCCCOC(=O)C1=CC=CC=C1C(=O)OCC2=CC=CC=C2,Active,active antagonist,inactive,-39.4007,,<rdkit.Chem.rdchem.Mol object at 0x7e9713537d80>,312.36,4.00,7,...,0,0,0,0,0,0,0,1,0,10.130435


In [57]:
data_descr.to_csv('molecular_descriptors.csv', index = False)